In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import bs4 as bs4
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import urllib

In [55]:
fromChat = pd.read_csv(r'/Users/krichnorakhun/Documents/GP/inventory/chatToyota.csv')
fromChat=fromChat[['Code']]
fromChat.columns = ['PartNO']
fromChat['Chat'] = 1

In [56]:
#Frist batch
GPInventory = pd.read_excel(r'/Users/krichnorakhun/Documents/GP/inventory/GP.xlsx')
GPInventory = GPInventory.loc[GPInventory['ItemGroup'].apply(lambda x : str(x).find('TOYOTA'))>=0][['ItemCode']]
GPInventory = GPInventory.drop_duplicates()
GPInventory.loc[:,'ItemCode'] = GPInventory['ItemCode'].apply(lambda x : x[4:] )
GPInventory = GPInventory.loc[GPInventory['ItemCode'].apply(lambda x : len(x)).isin([10,12])]
GPInventory.columns = ['Code']
GPInventory = GPInventory.reset_index()
GPInventory = GPInventory.drop(['index'], axis=1)
GPInventory.columns = ['PartNO']
GPInventory['Done'] = 1

In [57]:
NewFromGos = pd.read_excel(r'/Users/krichnorakhun/Documents/GP/inventory/List of Required EPC/Item-Toyota.xlsx', sheet_name='TOYOTA')
NewFromGos['PartNO'] = NewFromGos['ItemCode'].apply(lambda x : x.replace('TOY-',''))
NewFromGos = NewFromGos.loc[NewFromGos['PartNO'].apply(lambda x : len(x))>=10]
NewFromGos = NewFromGos[['PartNO']]
NewFromGos['Gos'] = 1

In [59]:
Comb = pd.merge( NewFromGos, fromChat, left_on='PartNO', right_on = 'PartNO', how='outer')

In [61]:
Comb = pd.merge( Comb,GPInventory, left_on='PartNO', right_on='PartNO', how='outer' )

In [80]:
Comb.loc[(Comb['Gos'].isna()) & (Comb['Done'].isna()) & (~Comb['Chat'].isna())]

,PartNO,Gos,Chat,Done
34805,6480102908,NaN,1.0,NaN
34806,8850128360,NaN,1.0,NaN
34807,8851528200,NaN,1.0,NaN
34808,87139YZZ45,NaN,1.0,NaN
34809,821540K010,NaN,1.0,NaN
...,...,...,...,...
46491,6168104020,NaN,1.0,NaN
46492,6810106111,NaN,1.0,NaN
46493,8266335050,NaN,1.0,NaN
46494,7735021020,NaN,1.0,NaN


In [87]:
Comb.to_csv(r'/Users/krichnorakhun/Documents/GP/inventory/TOY-CompleteList.csv', index=False)

In [86]:
Comb['PartNO'].apply(lambda x : len(x)).value_counts()

PartNO
10    29512
12    16250
11      698
13       45
14       24
15        1
Name: count, dtype: int64